In [21]:
library(tidyverse)
all_data = read_csv("/Users/dddd1007/project2git/cognitive_control_model/data/input/all_data_with_bl_rl_model_full.csv")
all_data_without_error <- all_data %>%
                            filter(Type == "hit")
sub_num_list <- unique(all_data_without_error$Subject_num)
glimpse(all_data_without_error)

Rows: 33600 Columns: 40

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): stim_color, stim_loc, Type, Subject, logfile_name, condition, fact...
dbl (31): Response, RT, Trial, prop, block, time_stamp, onset, Subject_num, ...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



Rows: 32,004
Columns: 40
$ stim_color       <chr> "red", "red", "red", "red", "green", "green", "green"…
$ stim_loc         <chr> "right", "left", "left", "left", "right", "right", "r…
$ Type             <chr> "hit", "hit", "hit", "hit", "hit", "hit", "hit", "hit…
$ Response         <dbl> 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,…
$ RT               <dbl> 939.5, 682.6, 391.8, 404.8, 369.6, 392.5, 325.8, 491.…
$ Subject          <chr> "sub01_Yangmiao", "sub01_Yangmiao", "sub01_Yangmiao",…
$ Trial            <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16…
$ logfile_name     <chr> "v_80", "v_80", "v_80", "v_80", "v_80", "v_80", "v_80…
$ prop             <dbl> 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 8…
$ condition        <chr> "v", "v", "v", "v", "v", "v", "v", "v", "v", "v", "v"…
$ block            <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ time_stamp       <dbl> 1553158380, 1553158380, 1553158380, 1553158380, 15531…
$ onset        

In [22]:
# 根据反应时比较 AIC
aic_bl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_bl_sr   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_sr   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab_v <- vector(mode = "numeric", length(sub_num_list))

for (i in seq_len(length(sub_num_list))) {
    single_sub_data <- all_data_without_error %>%
                        filter(Subject_num == sub_num_list[i])
    aic_bl_ab[i]   <- AIC(lm(RT ~ bl_ab_r_selected, data = single_sub_data))
    aic_bl_sr[i]   <- AIC(lm(RT ~ bl_sr_r_selected, data = single_sub_data))
    aic_rl_ab[i]   <- AIC(lm(RT ~ rl_ab_p, data = single_sub_data))
    aic_rl_sr[i]   <- AIC(lm(RT ~ rl_sr_p, data = single_sub_data))
    aic_rl_ab_v[i] <- AIC(lm(RT ~ rl_ab_v_p, data = single_sub_data))
}

RT_aic <- data.frame(Subject_num = sub_num_list,
                     bl_ab = aic_bl_ab,
                     bl_sr = aic_bl_sr,
                     rl_ab = aic_rl_ab,
                     rl_sr = aic_rl_sr,
                     rl_ab_v = aic_rl_ab_v)


In [23]:
RT_aic

Subject_num,bl_ab,bl_sr,rl_ab,rl_sr,rl_ab_v
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,11384.880,11391.466,11412.26,11434.403,11412.165
2,11265.668,11261.614,11245.52,11291.768,11246.325
3,11032.774,11033.229,11033.66,11031.903,11033.462
4,11628.676,11629.175,11637.55,11638.648,11637.736
5,12565.297,12565.202,12569.68,12566.807,12570.212
6,7504.914,7503.666,7505.33,7504.858,7505.195
7,11604.103,11606.512,11633.20,11627.096,11633.091
8,11203.761,11208.392,11216.50,11221.037,11217.461
9,11443.534,11442.464,11446.97,11448.892,11446.465


In [27]:
# 和刺激特征的比较
aic_bl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab_v <- vector(mode = "numeric", length(sub_num_list))
aic_rl_sr   <- vector(mode = "numeric", length(sub_num_list))

for (i in seq_len(length(sub_num_list))) {
    single_sub_data <- all_data_without_error %>%
                        filter(Subject_num == sub_num_list[i])
    aic_bl_ab[i]   <- AIC(glm(congruency_num ~ bl_ab_r_selected, data = single_sub_data, family = binomial))
    aic_rl_ab[i]   <- AIC(glm(congruency_num ~ rl_ab_p, data = single_sub_data, family = binomial))
    aic_rl_ab_v[i] <- AIC(glm(congruency_num ~ rl_ab_v_p, data = single_sub_data, family = binomial))
    aic_rl_sr[i]   <- AIC(glm(correct_action ~ rl_sr_p, data = single_sub_data, family = binomial))
}

stim_aic <- data.frame(Subject_num = sub_num_list,
                     bl_ab = aic_bl_ab,
                     rl_ab = aic_rl_ab,
                     rl_ab_v = aic_rl_ab_v,
                     rl_sr = aic_rl_sr)

In [28]:
stim_aic

Subject_num,bl_ab,rl_ab,rl_ab_v,rl_sr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1323.6256,1040.3599,1036.2047,1117.9316
2,1301.4978,1055.3624,1054.7455,1122.7672
3,1155.6957,957.7847,956.9581,1008.1865
4,1313.9673,1030.6246,1027.8870,1101.0166
5,1319.5883,1097.7848,1095.9584,1166.4036
6,733.5663,581.9030,581.7248,571.1833
7,1324.9387,1074.4708,1070.6633,1140.0686
8,1171.2041,973.7537,971.5625,1023.9569
9,1302.7544,1062.1058,1055.6636,1132.8248
